# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('../starter_files/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'ML RDCEQL S/N KTALTABAEGA)S(OENGS',
 'calculationPrice': 'close',
 'open': 123.64,
 'openTime': 1630238348621,
 'openSource': 'olfiifca',
 'close': 122.41,
 'closeTime': 1693282276660,
 'closeSource': 'lfiafcio',
 'high': 127.201,
 'highTime': 1645079999116,
 'highSource': 'e5eduli e1nc ade prmyit',
 'low': 121.78,
 'lowTime': 1669980254254,
 'lowSource': 'pm5d e cnutileeae1 diyr',
 'latestPrice': 124.9,
 'latestSource': 'Close',
 'latestTime': 'March 5, 2021',
 'latestUpdate': 1668737762129,
 'latestVolume': 158899022,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 123.3,
 'delayedPriceTime': 1659592598865,
 'oddLotDelayedPrice': 124.84,
 'oddLotDelayedPriceTime': 1688663017753,
 'extendedPrice': 127.46,
 'extendedChange': 0.07,
 'extendedChangePercent': 0.0006,
 'extendedPriceTime': 1633062979919,
 'previousClose': 121.47,
 'previousVolume': 183039373,
 'change': 1.3,


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

33.37

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,119.32,46.64,N/A
1,AAL,20.74,-1.16,N/A
2,AAP,179.70,24.15,N/A
3,AAPL,127.42,34.57,N/A
4,ABBV,111.40,40.2,N/A
...,...,...,...,...
500,YUM,108.97,35.73,N/A
501,ZBH,164.90,-236.49,N/A
502,ZBRA,497.76,52.23,N/A
503,ZION,57.14,8.81,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,569.71,4.6,351
1,AIV,5.04,4.82,39682
2,ALL,116.35,6.7,1718
3,EBAY,55.87,7.06,3579
4,UNM,28.85,7.7,6932
5,AFL,51.98,7.7,3847
6,CPB,48.28,8.31,4142
7,ZION,57.14,8.81,3500
8,KIM,19.64,8.84,10183
9,CE,150.34,8.98,1330


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,234.31,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
42,APA,23.72,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,258.25,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,10.15,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,54.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.23,N/A,14.88,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,56.40,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.73,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,170.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2187.94,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0        0.827723
1        0.154455
2        0.469307
3         0.69802
4        0.780198
          ...    
500      0.734653
501    0.00990099
502      0.855446
503      0.180198
504       0.80396
Name: PE Percentile, Length: 505, dtype: object
0       0.728713
1      0.0455446
2       0.418812
3       0.954455
4       0.880198
         ...    
500    0.0415842
501     0.372277
502     0.851485
503     0.134653
504     0.917822
Name: PB Percentile, Length: 505, dtype: object
0       0.764356
1      0.0653465
2       0.133663
3       0.805941
4            0.6
         ...    
500     0.733663
501     0.669307
502     0.746535
503      0.39802
504     0.912871
Name: PS Percentile, Length: 505, dtype: object
0       0.845545
1       0.039604
2       0.251485
3       0.770297
4            0.4
         ...    
500     0.758416
501     0.865347
502     0.891089
503    0.0752475
504     0.869307
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.752475
1      0.0871287
2       

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.02,N/A,46.88,0.827723,7.43,0.728713,6.4000,0.764356,28.729193,0.845545,12.227802,0.752475,N/A
1,AAL,20.68,N/A,-1.16,0.154455,-2.00,0.0455446,0.7649,0.0653465,-4.246145,0.039604,2.206424,0.0871287,N/A
2,AAP,177.57,N/A,24.60,0.469307,3.20,0.418812,1.1500,0.133663,11.463998,0.251485,2.515961,0.106931,N/A
3,AAPL,125.75,N/A,34.13,0.69802,31.70,0.954455,7.1300,0.805941,24.203221,0.770297,18.784644,0.920792,N/A
4,ABBV,109.80,N/A,40.88,0.780198,14.42,0.880198,4.2900,0.6,14.561497,0.4,8.858327,0.544554,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.12,N/A,36.56,0.734653,-4.09,0.0415842,5.8000,0.733663,23.652122,0.758416,9.977579,0.663366,N/A
501,ZBH,165.40,N/A,-247.97,0.00990099,2.80,0.372277,4.8800,0.669307,29.874966,0.865347,8.368407,0.506931,N/A
502,ZBRA,491.10,N/A,52.32,0.855446,12.24,0.851485,5.9400,0.746535,35.120763,0.891089,13.302180,0.792079,N/A
503,ZION,56.66,N/A,9.21,0.180198,1.31,0.134653,2.6800,0.39802,6.959236,0.0752475,2.937113,0.124752,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.02,N/A,46.88,0.827723,7.43,0.728713,6.4000,0.764356,28.729193,0.845545,12.227802,0.752475,0.783762
1,AAL,20.68,N/A,-1.16,0.154455,-2.00,0.0455446,0.7649,0.0653465,-4.246145,0.039604,2.206424,0.0871287,0.0784158
2,AAP,177.57,N/A,24.60,0.469307,3.20,0.418812,1.1500,0.133663,11.463998,0.251485,2.515961,0.106931,0.27604
3,AAPL,125.75,N/A,34.13,0.69802,31.70,0.954455,7.1300,0.805941,24.203221,0.770297,18.784644,0.920792,0.829901
4,ABBV,109.80,N/A,40.88,0.780198,14.42,0.880198,4.2900,0.6,14.561497,0.4,8.858327,0.544554,0.64099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.12,N/A,36.56,0.734653,-4.09,0.0415842,5.8000,0.733663,23.652122,0.758416,9.977579,0.663366,0.586337
501,ZBH,165.40,N/A,-247.97,0.00990099,2.80,0.372277,4.8800,0.669307,29.874966,0.865347,8.368407,0.506931,0.484752
502,ZBRA,491.10,N/A,52.32,0.855446,12.24,0.851485,5.9400,0.746535,35.120763,0.891089,13.302180,0.792079,0.827327
503,ZION,56.66,N/A,9.21,0.180198,1.31,0.134653,2.6800,0.39802,6.959236,0.0752475,2.937113,0.124752,0.182574


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_input()

Enter the value of your portfolio:1000000


In [22]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\dhoch\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.61,675,7.60,0.168317,0.5638,0.0534653,0.4507,0.0217822,3.694434,0.0475248,0.447908,0.0039604,0.0590099
1,MCK,179.64,111,-7.17,0.114851,-59.6900,0.0039604,0.1212,0.00594059,6.923688,0.0732673,2.520901,0.108911,0.0613861
2,FTI,9.35,2139,-0.74,0.157426,1.0210,0.0831683,0.3133,0.0118812,2.745223,0.0455446,1.796854,0.0574257,0.0710891
3,AIG,46.98,425,-6.94,0.118812,0.6445,0.0554455,0.9299,0.0831683,7.482475,0.0891089,0.941867,0.0217822,0.0736634
4,AAL,20.68,967,-1.16,0.154455,-2.0000,0.0455446,0.7649,0.0653465,-4.246145,0.039604,2.206424,0.0871287,0.0784158
5,MET,61.59,324,10.67,0.194059,0.7325,0.0574257,0.7997,0.0712871,5.287786,0.0534653,0.766008,0.0178218,0.0788119
6,ALL,112.95,177,6.67,0.164356,1.2000,0.10297,0.7702,0.0673267,2.496068,0.0435644,0.761516,0.0158416,0.0788119
7,ABC,109.10,183,-6.68,0.120792,-31.7000,0.0118812,0.1144,0.0039604,7.702168,0.0950495,3.730624,0.168317,0.08
8,F,12.79,1563,-39.64,0.049505,1.5800,0.178218,0.3948,0.0178218,6.069094,0.0594059,2.801127,0.120792,0.0851485
9,DXC,28.55,700,-2.48,0.141584,1.2700,0.117822,0.3958,0.019802,5.470240,0.0554455,2.461052,0.10297,0.0875248


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [26]:
writer.save()